In [4]:
#Import libraries
import numpy as np
import pandas as pd
import torch
import os
import glob

In [13]:
#Old baseline function for testing: using a single spreadsheet
def baseline_old(neighbourhood,housing_type,guest_num):
    #Reading csv files
    data = pd.read_csv("../baseline_data/18_04_09_listings.csv")
    neighbourhood_reference = pd.read_csv("../category_codes/neighbourhoods.csv")
    
    #Finding corresponding numerical values based on neighbourhood
    district_group = neighbourhood_reference.loc[neighbourhood_reference['neighbourhood'] == neighbourhood, 'district_group'].iloc[0]
    district_number = neighbourhood_reference.loc[neighbourhood_reference['neighbourhood'] == neighbourhood, 'district_number'].iloc[0]
    neighbourhood_value = district_group*15 + district_number
    
    #Identifying rows of interest
    property_type = data['room_type'] == housing_type
    accomodates = data['accommodates'] == guest_num
    district = data['district'] == neighbourhood_value
    interested = data[property_type & accomodates & district]
    
    #Computing average
    average = interested['price'].mean()
    
    #Printing the average
    print('the average is',average)
    
    #Converting column to list
    l = interested['price'].tolist()
    
    #Computing standard deviation
    difference = [(x - average) for x in l]
    squared = [x**2 for x in difference]
    summation = sum(squared)
    std = (summation / len(l))**0.5
    
    #Printing
    #print(std)
    
    #Returning
    return [std,average]

    
#Testing
#baseline('South Parkdale','Entire home/apt',2)
    

the average is 95.78813559322033
[125, 100, 125, 55, 120, 75, 75, 72, 194, 54, 45, 135, 95, 86, 86, 150, 196, 72, 55, 65, 50, 75, 59, 72, 52, 60, 70, 81, 75, 130, 100, 75, 89, 175, 130, 118, 98, 70, 110, 86, 150, 65, 81, 199, 63, 70, 144, 63, 65, 100, 165, 106, 50, 175, 69, 75, 81, 95, 100, 68, 100, 70, 86, 89, 79, 150, 100, 95, 75, 65, 81, 110, 81, 79, 73, 105, 79, 64, 86, 150, 70, 150, 130, 54, 110, 60, 50, 81, 129, 190, 95, 100, 125, 86, 89, 86, 81, 75, 89, 65, 79, 70, 66, 69, 100, 251, 86, 95, 69, 86, 100, 115, 120, 175, 157, 77, 65, 52]
37.86883063697799


95.78813559322033

In [26]:
#baseline model helper function, for obtaining price given specific features and standard deviation
def baseline(neighbourhood,housing_type,guest_num,data):
    
    #Reading csv files
    neighbourhood_reference = pd.read_csv("../category_codes/neighbourhoods.csv")
    
    #Finding corresponding numerical values based on neighbourhood
    district_group = neighbourhood_reference.loc[neighbourhood_reference['neighbourhood'] == neighbourhood, 'district_group'].iloc[0]
    district_number = neighbourhood_reference.loc[neighbourhood_reference['neighbourhood'] == neighbourhood, 'district_number'].iloc[0]
    neighbourhood_value = district_group*15 + district_number
    
    #Identifying rows of interest
    property_type = data['room_type'] == housing_type
    accomodates = data['accommodates'] == guest_num
    district = data['district'] == neighbourhood_value
    interested = data[property_type & accomodates & district]
    
    #Computing average
    average = interested['price'].mean()
    
    #Printing the average
    print('The average is',average)
    
    #Converting column to list
    l = interested['price'].tolist()
    
    #Computing standard deviation
    difference = [(x - average) for x in l]
    squared = [x**2 for x in difference]
    summation = sum(squared)
    std = -1
    #Checking if the length of list is 0 and computing standard deviation
    if len(l) > 0:
        std = (summation / len(l))**0.5
    
    #Printing
    print('The standard deviation is:', std)
    
    #Returning
    return [std,average]

In [27]:
def baseline_accuracy():
    #Reading csv files from given folder
    path = r'../baseline_data'
    all_files = glob.glob(path + "/*.csv")
    li = []
    std_list = []
    average_list = []
    for filename in all_files:
        df = pd.read_csv(filename,index_col = None, header = 0)
        li.append(df)
    
    #Concatenating all the dataframes into a single frame
    frame = pd.concat(li,axis=0,ignore_index = True)
    
    #Loading the neighbourhood reference spreadsheet
    neighbourhood_reference = pd.read_csv("../category_codes/neighbourhoods.csv")
    
    #Finding the possible values for room_type, accomodates, and neighbourhood features.
    unique_roomtype_values = frame['room_type'].unique()
    unique_accomodates_values = frame['accommodates'].unique()
    unique_neighbourhood_values = neighbourhood_reference['neighbourhood'].unique()
    
    #Checking the unique values by printing
    #print('unique_roomtype_values', unique_roomtype_values)
    #print('unique_accomodates_values', unique_accomodates_values)
    #print('unique_neighbourhood_values',unique_neighbourhood_values)
    
    
    #Computing the various combinations of room_type, accomodates, and neighbourhood
    for i in range (0, len(unique_roomtype_values)):
        for j in range (0, len(unique_accomodates_values)):
            for k in range (0, len(unique_neighbourhood_values)):
                neighbourhood = unique_neighbourhood_values[k]
                housing_type = unique_roomtype_values[i]
                guest_num = unique_accomodates_values[j]
                
                #Printing the combinations
                print('-------------')
                print('Room type:', unique_roomtype_values[i])
                print('Number of people accommodated:', unique_accomodates_values[j])
                print('Neighbourhood:',unique_neighbourhood_values[k])
                
                #Calling the helper function to compute standard deviation and average
                [std,average] = baseline(neighbourhood,housing_type,guest_num,frame)
                if std != -1:
                    average_list.append(average)
                    std_list.append(std)
    
    #Finding the total standard deviation and average standard deviation
    total_std = sum(std_list)
    average_error = total_std/len(std_list)
    
    #Printing results
    print('Total Standard Deviation:', total_std)
    print('Total Valid Combinations:', len(std_list))
    print('Baseline average standard deviation:',average_error)
    
    #Returning
    return average_error
                
                
    
  
error = baseline_accuracy()

    
    

-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Agincourt North
The average is 50.78279386712095
The standard deviation is: 51.28978376440816
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Agincourt South-Malvern West
The average is 50.78279386712095
The standard deviation is: 51.28978376440816
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Alderwood
The average is 69.62360540223136
The standard deviation is: 56.81125186836449
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Annex
The average is 69.17007874015748
The standard deviation is: 29.607146885726035
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Banbury-Don Mills
The average is 66.84623323013416
The standard deviation is: 61.919253061064246
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Bathurst Mano

The average is 60.64074803149607
The standard deviation is: 27.43413502909808
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Glenfield-Jane Heights
The average is 45.499376558603494
The standard deviation is: 15.476371648801223
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Greenwood-Coxwell
The average is 70.47940074906367
The standard deviation is: 32.47359265411852
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Guildwood
The average is 50.82093023255814
The standard deviation is: 19.768733336744628
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Henry Farm
The average is 58.74462910417511
The standard deviation is: 45.89000576639998
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Highland Creek
The average is 61.97785977859779
The standard deviation is: 28.985864168913693
-------------
Room ty

The standard deviation is: 48.09113804329293
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Oakridge
The average is 49.524079320113316
The standard deviation is: 17.18785107914661
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Oakwood Village
The average is 60.64074803149607
The standard deviation is: 27.43413502909808
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: O'Connor-Parkview
The average is 60.231155778894475
The standard deviation is: 21.400558322402503
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Old East York
The average is 60.231155778894475
The standard deviation is: 21.400558322402503
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Palmerston-Little Italy
The average is 66.69501054111034
The standard deviation is: 24.85431387472416
-------------
Room type: Private room
Number of p

The standard deviation is: 21.400558322402503
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Wychwood
The average is 69.17007874015748
The standard deviation is: 29.607146885726035
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Yonge-Eglinton
The average is 60.64074803149607
The standard deviation is: 27.43413502909808
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Yonge-St.Clair
The average is 69.17007874015748
The standard deviation is: 29.607146885726035
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: Yorkdale-Glen Park
The average is 69.62360540223136
The standard deviation is: 56.81125186836449
-------------
Room type: Private room
Number of people accommodated: 2
Neighbourhood: York University Heights
The average is 45.499376558603494
The standard deviation is: 15.476371648801223
-------------
Room type: Private room
Number of 

The average is 53.88063660477454
The standard deviation is: 25.396465252867227
-------------
Room type: Private room
Number of people accommodated: 1
Neighbourhood: Eringate-Centennial-West Deane
The average is 55.66350710900474
The standard deviation is: 23.372839282727423
-------------
Room type: Private room
Number of people accommodated: 1
Neighbourhood: Etobicoke West Mall
The average is 55.66350710900474
The standard deviation is: 23.372839282727423
-------------
Room type: Private room
Number of people accommodated: 1
Neighbourhood: Flemingdon Park
The average is 49.05978260869565
The standard deviation is: 24.057488031536526
-------------
Room type: Private room
Number of people accommodated: 1
Neighbourhood: Forest Hill North
The average is 53.88063660477454
The standard deviation is: 25.396465252867227
-------------
Room type: Private room
Number of people accommodated: 1
Neighbourhood: Forest Hill South
The average is 61.59375
The standard deviation is: 47.91445275035573
---

...

Number of people accommodated: 13
Neighbourhood: Cabbagetown-South St.James Town
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Caledonia-Fairbank
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Casa Loma
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Centennial Scarborough
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Church-Yonge Corridor
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Clairlea-Birchmount
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Clanton Park
Th

The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Malvern
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Maple Leaf
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Markland Wood
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Milliken
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Mimico (includes Humber Bay Shores)
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Morningside
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of peop

Neighbourhood: Weston
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Weston-Pellam Park
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Wexford/Maryvale
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Willowdale East
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Willowdale West
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Willowridge-Martingrove-Richview
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 13
Neighbourhood: Woburn
The average is nan
The standard deviation is: -1
-----------

-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Eringate-Centennial-West Deane
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Etobicoke West Mall
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Flemingdon Park
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Forest Hill North
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Forest Hill South
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Glenfield-Jane Heights
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodat

The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Palmerston-Little Italy
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Parkwoods-Donalda
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Pelmo Park-Humberlea
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Playter Estates-Danforth
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Pleasant View
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Number of people accommodated: 14
Neighbourhood: Princess-Rosethorn
The average is nan
The standard deviation is: -1
-------------
Room type: Shared room
Nu

In [25]:
def baseline_accuracy():
    #Reading csv files from given folder
    path = r'../baseline_data'
    all_files = glob.glob(path + "/*.csv")
    li = []
    std_list = []
    average_list = []
    for filename in all_files:
        df = pd.read_csv(filename,index_col = None, header = 0)
        li.append(df)
    
    #Concatenating all the dataframes into a single frame
    frame = pd.concat(li,axis=0,ignore_index = True)
    
    #Loading the neighbourhood reference spreadsheet
    neighbourhood_reference = pd.read_csv("../category_codes/neighbourhoods.csv")
    
    #Finding the possible values for room_type, accomodates, and neighbourhood features.
    unique_roomtype_values = frame['room_type'].unique()
    unique_accomodates_values = frame['accommodates'].unique()
    unique_neighbourhood_values = neighbourhood_reference['neighbourhood'].unique()
    
    #Checking the unique values by printing
    #print('unique_roomtype_values', unique_roomtype_values)
    #print('unique_accomodates_values', unique_accomodates_values)
    #print('unique_neighbourhood_values',unique_neighbourhood_values)
    
    
    #Computing the various combinations of room_type, accomodates, and neighbourhood
    for i in range (0, len(unique_roomtype_values)):
        for j in range (0, len(unique_accomodates_values)):
            for k in range (0, len(unique_neighbourhood_values)):
                neighbourhood = unique_neighbourhood_values[k]
                housing_type = unique_roomtype_values[i]
                guest_num = unique_accomodates_values[j]
                
                #Printing the combinations
                print('-------------')
                print('Room type:', unique_roomtype_values[i])
                print('Number of people accommodated:', unique_accomodates_values[j])
                print('Neighbourhood:',unique_neighbourhood_values[k])
                
                #Calling the helper function to compute standard deviation and average
                [std,average] = baseline(neighbourhood,housing_type,guest_num,frame)
                if std != -1:
                    average_list.append(average)
                    std_list.append(std)
    
    #Finding the total standard deviation and average standard deviation
    total_std = sum(std_list)
    average_error = total_std/len(std_list)
    
    #Printing results
    print('Total Standard Deviation:', total_std)
    print('Total Valid Combinations:', len(std_list))
    print('Baseline average standard deviation:',average_error)
    
    #Returning
    return average_error
                
                
    
  
error = baseline_accuracy()


Total Standard Deviation: 156208.64908889396
Total Valid Combinations: 3228
Baseline average standard deviation: 48.39177481068586


'\n    #data = pd.read_csv("../processed_data/18_04_09_listings.csv")\n    neighbourhood_reference = pd.read_csv("../category_codes/neighbourhoods.csv")\n    \n    #Finding corresponding numerical values based on neighbourhood\n    district_group = neighbourhood_reference.loc[neighbourhood_reference[\'neighbourhood\'] == neighbourhood, \'district_group\'].iloc[0]\n    district_number = neighbourhood_reference.loc[neighbourhood_reference[\'neighbourhood\'] == neighbourhood, \'district_number\'].iloc[0]\n    neighbourhood_value = district_group*15 + district_number\n    \n    #Identifying rows of interest\n    property_type = data[\'room_type\'] == housing_type\n    accomodates = data[\'accommodates\'] == guest_num\n    district = data[\'district\'] == neighbourhood_value\n    interested = data[property_type & accomodates & district]\n    \n    #Computing average\n    average = interested[\'price\'].mean()\n    \n    #Printing the average\n    print(\'the average is\',average)\n    \n   